In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm

from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

import pylab as pl
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('KaggleV1_with_weather_dec_16_1113.csv')

In [11]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,maxdewptm,meantempi,date_tzname,date_mon,date_hour,date_min,date_year,date_pretty,date_mday,datetime
0,2.987250e+13,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,...,20.3,72.5,America/Sao_Paulo,4.0,0.0,0.0,2016.0,"April 29, 2016",29.0,2016-04-29
1,5.589980e+14,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,...,20.3,72.5,America/Sao_Paulo,4.0,0.0,0.0,2016.0,"April 29, 2016",29.0,2016-04-29
2,4.262960e+12,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,...,20.3,72.5,America/Sao_Paulo,4.0,0.0,0.0,2016.0,"April 29, 2016",29.0,2016-04-29
3,8.679510e+11,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,...,20.3,72.5,America/Sao_Paulo,4.0,0.0,0.0,2016.0,"April 29, 2016",29.0,2016-04-29
4,8.841190e+12,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,...,20.3,72.5,America/Sao_Paulo,4.0,0.0,0.0,2016.0,"April 29, 2016",29.0,2016-04-29


In [7]:
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show',
       'time_d', 'time_d_total_seconds', 'a_day_of_week', 's_day_of_week',
       'Neighbourhood_category', 'Gender_category', 'no_show',
       'scheduled_hour', 'time_d_total_days', 'minpressurem', 'minhumidity',
       'maxtempi', 'mintempi', 'maxpressurei', 'precipm', 'mindewptm',
       'meanwindspdm', 'meandewpti', 'mintempm', 'maxhumidity', 'maxpressurem',
       'meantempm', 'precipi', 'meanwdire', 'maxwspdi', 'humidity',
       'minpressurei', 'maxdewpti', 'maxtempm', 'mindewpti', 'meandewptm',
       'meanwindspdi', 'meanwdird', 'maxwspdm', 'maxdewptm', 'meantempi',
       'date_tzname', 'date_mon', 'date_hour', 'date_min', 'date_year',
       'date_pretty', 'date_mday', 'datetime'],
      dtype='object')

In [8]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,time_d_total_seconds,...,meanwindspdi,meanwdird,maxwspdm,maxdewptm,meantempi,date_mon,date_hour,date_min,date_year,date_mday
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,1.105270e+05,...,106248.000000,0.0,106248.000000,106248.000000,106248.000000,106248.000000,106248.0,106248.0,106248.0,106248.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026,8.393142e+05,...,6.573320,NaN,27.276469,21.719275,76.006910,5.218508,0.0,0.0,2016.0,11.754311
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873,1.317088e+06,...,2.782626,NaN,8.354931,1.589639,2.772242,0.481310,0.0,0.0,0.0,8.425158
min,3.920000e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.681600e+05,...,3.100000,NaN,12.900000,18.100000,71.500000,4.000000,0.0,0.0,2016.0,1.000000
25%,4.172615e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.990850e+04,...,4.100000,NaN,20.900000,20.800000,73.600000,5.000000,0.0,0.0,2016.0,5.000000
50%,3.173180e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.001380e+05,...,6.000000,NaN,27.400000,21.900000,75.600000,5.000000,0.0,0.0,2016.0,10.000000
75%,9.439170e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.237294e+06,...,8.400000,NaN,33.800000,22.800000,78.100000,5.000000,0.0,0.0,2016.0,18.000000
max,9.999820e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.542714e+07,...,14.400000,NaN,41.800000,24.900000,82.900000,6.000000,0.0,0.0,2016.0,30.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 58 columns):
PatientId                 110527 non-null float64
AppointmentID             110527 non-null int64
Gender                    110527 non-null object
ScheduledDay              110527 non-null object
AppointmentDay            110527 non-null object
Age                       110527 non-null int64
Neighbourhood             110527 non-null object
Scholarship               110527 non-null int64
Hipertension              110527 non-null int64
Diabetes                  110527 non-null int64
Alcoholism                110527 non-null int64
Handcap                   110527 non-null int64
SMS_received              110527 non-null int64
No-show                   110527 non-null object
time_d                    110527 non-null object
time_d_total_seconds      110527 non-null float64
a_day_of_week             110527 non-null int64
s_day_of_week             110527 non-null int64
Neighbourhood_c

In [10]:
df.isnull().sum()

PatientId                      0
AppointmentID                  0
Gender                         0
ScheduledDay                   0
AppointmentDay                 0
Age                            0
Neighbourhood                  0
Scholarship                    0
Hipertension                   0
Diabetes                       0
Alcoholism                     0
Handcap                        0
SMS_received                   0
No-show                        0
time_d                         0
time_d_total_seconds           0
a_day_of_week                  0
s_day_of_week                  0
Neighbourhood_category         0
Gender_category                0
no_show                        0
scheduled_hour                 0
time_d_total_days              0
minpressurem                4279
minhumidity                 4279
maxtempi                    4279
mintempi                    4279
maxpressurei                4279
precipm                     4279
mindewptm                   4279
meanwindsp